# Plot: Global Fine Air Particulates PM2.5 2010-2019

Use Dash to plot 3 interactive graphs
- Line chart: PM2.5 - Year / All countries, colour coded by continents, WHO guidelines

- Line chart: PM2.5 - Year / by country, 2010-2019 dataset (mean, max, min), regression-prediction, actual 2020/2021/2022
- tbc

Dataset:mean_pm25.csv
Good to have: NO2

In [21]:
import dash
from dash import dcc, html, callback, Output, Input
import plotly.express as px
import pandas as pd

# Import data
df_mean = pd.read_csv("pm25_mean.csv")  
df_min = pd.read_csv("pm25_min.csv")  
df_max = pd.read_csv("pm25_max.csv")  


# Define columns
years = df_mean.columns[2:12].tolist()
countries = df_mean.iloc[1:192, 0].tolist()
continents = df_mean.iloc[1:192, 1].tolist()

pm_mean = df_mean.iloc[1:192, 2:12].values.tolist()
pm_min = df_min.iloc[1:192, 2:12].values.tolist()
pm_max = df_max.iloc[1:192, 2:12].values.tolist()


app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in countries],
            value=countries[0]
        )
    ]),
    html.Div([
        dcc.Graph(id='pm-line-chart')
    ])
])

@app.callback(
    Output('pm-line-chart', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_line_chart(selected_country):
    country_index = countries.index(selected_country)
    selected_pm_mean = pm_mean[country_index]
    selected_pm_min = pm_min[country_index]
    selected_pm_max = pm_max[country_index]

    fig = px.line(x=years, y=selected_pm_mean, labels={'y':'PM2.5 (µg/m³)', }, title=f'Amount of fine air particles smaller than 2.5 mircometers: {selected_country}')
    fig.update_yaxes(range=[0,115])
    fig.update_traces(name='Mean')

    fig.add_scatter(x=years, y=selected_pm_max, mode='lines', name='Max')
    fig.add_scatter(x=years, y=selected_pm_min, mode='lines', name='Min')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=4000)
